# CPI inflation graphic
Date: *March 16, 2022*

A quick tutorial on how I prepared a graphic on CPI inflation for CBC News. We import Pandas, then we import the data.

In [21]:
import pandas as pd

raw = pd.read_csv('https://www150.statcan.gc.ca/t1/tbl1/en/dtl!downloadDbLoadingData-nonTraduit.action?pid=1810000401&latestN=0&startDate=19140101&endDate=20221201&csvLocale=en&selectedMembers=%5B%5B2%5D%2C%5B2%2C3%2C79%2C96%2C139%2C176%2C184%2C201%2C219%2C256%2C274%2C282%2C285%2C287%2C288%5D%5D&checkedLevels=')

raw.head()

,REF_DATE,GEO,DGUID,Products and product groups,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1914-01,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.2,6.0,NaN,NaN,NaN,1
1,1914-02,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.2,6.0,NaN,NaN,NaN,1
2,1914-03,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.2,6.0,NaN,NaN,NaN,1
3,1914-04,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.2,6.0,NaN,NaN,NaN,1
4,1914-05,Canada,2016A000011124,All-items,2002=100,17,units,0,v41690973,2.2,5.9,NaN,NaN,NaN,1


First we have to filter the data to show only what we want to see: all basket items, and only for Canada (not provinces or territories). We also filter for data from 2000 onwards.

In [22]:
filtered = raw[(raw["Products and product groups"] == "All-items")
               & (raw["GEO"] == "Canada")
               & (raw["REF_DATE"] >= "2000-01")
               ]

filtered = (filtered
            .pivot(index="REF_DATE", values="VALUE", columns="Products and product groups")
            .reset_index()
            )

Now we have to work some magic on the data. We'll start by calculating the % change vs. the same month in the previous year. Then, we get rid of empty values (really, just the first 12 values because they can't compare to anything!).

In [23]:
filtered["change"] = (filtered["All-items"].pct_change(12) * 100)
filtered = filtered.dropna(subset=["change"])
filtered["REF_DATE"] = pd.to_datetime(filtered["REF_DATE"])

filtered

Products and product groups,REF_DATE,All-items,change
12,2001-01-01,96.3,2.994652
13,2001-02-01,96.8,2.869288
14,2001-03-01,97.1,2.426160
15,2001-04-01,97.8,3.492063
16,2001-05-01,98.6,3.898841
...,...,...,...
261,2021-10-01,143.9,4.654545
262,2021-11-01,144.2,4.720407
263,2021-12-01,144.0,4.803493
264,2022-01-01,145.3,5.137482


Using this set, we'll do two version and join them: one for monthly changes over the previous year, and the other for only February changes, with interpolations in between (so we get a nice smooth line).

In [26]:
annual = filtered[["REF_DATE", "change"]].set_index("REF_DATE")

annual.to_clipboard()

Now we take it over to Datawrapper, and here is the result: [https://www.datawrapper.de/_/LgM77/](https://www.datawrapper.de/_/LgM77/).